# Aviation Risk Prediction Intelligence — EDA (Exploratory Data Analysis)

This notenook analyse the enriched dataset that is the fusion between : 
- The real flights data (OpenSky Network) located in  `src/ingestion/opensky_ingest.py`
- Meteorological Airport Report data (METAR) located `src/ingestion/metar_ingest.py`

Objectives :
- Understand data structures
- detect anomalies
- identify flights and meteorological conditions patterns
- Prepare the feature engineering necessary for the Machine Learning

The analysis is done on the most recent enriched file available in `data/processed/`.

In [2]:
import os, sys

PROJECT_ROOT = os.path.abspath("..")
os.chdir(PROJECT_ROOT)
if PROJECT_ROOT not in sys.path:
    sys.path.append(PROJECT_ROOT)

print("Working directory set to:", os.getcwd())



Working directory set to: c:\Users\yassi\OneDrive\Documents\Administratif\Data_Science\aviation-risk-project


In [11]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime, timezone

from src.utils import get_latest_file

sns.set_theme(style="whitegrid")
plt.rcParams["figure.figsize"] = (12, 6)


In [4]:
pattern = "data/processed/flights_enriched_*.csv"

filepath = get_latest_file(pattern)

print("Loaded File:", filepath)

df=pd.read_csv(filepath)

df.head()

Loaded File: data/processed\flights_enriched_20251210_173141.csv


,icao24,callsign,origin_country,time_position,last_contact,longitude,latitude,baro_altitude,on_ground,velocity,...,metar_metarType,metar_rawOb,metar_lat,metar_lon,metar_elev,metar_name,metar_cover,metar_clouds,metar_fltCat,metar_fetch_time_utc
0,80162c,AXB522,India,1.765380e+09,1765379869,55.6938,24.0396,10668.00,False,251.49,...,METAR,METAR LFPG 101600Z 27006KT 9999 BKN030 BKN190 ...,49.015,2.534,107,"Paris/De Gaulle Arpt, ID, FR",BKN,"[{'cover': 'BKN', 'base': 3000}, {'cover': 'BK...",MVFR,2025-12-10 16:20:04.386928+00:00
1,801638,AXB1120,India,1.765380e+09,1765379869,77.9913,27.8394,8839.20,False,252.36,...,METAR,METAR LFPG 101600Z 27006KT 9999 BKN030 BKN190 ...,49.015,2.534,107,"Paris/De Gaulle Arpt, ID, FR",BKN,"[{'cover': 'BKN', 'base': 3000}, {'cover': 'BK...",MVFR,2025-12-10 16:20:04.386928+00:00
2,408120,VIR47GH,United Kingdom,1.765380e+09,1765379869,-3.3146,51.7957,8191.50,False,183.03,...,METAR,METAR LFPG 101600Z 27006KT 9999 BKN030 BKN190 ...,49.015,2.534,107,"Paris/De Gaulle Arpt, ID, FR",BKN,"[{'cover': 'BKN', 'base': 3000}, {'cover': 'BK...",MVFR,2025-12-10 16:20:04.386928+00:00
3,88044a,AIQ3228,Thailand,1.765380e+09,1765379870,100.0653,12.0238,11277.60,False,239.34,...,METAR,METAR LFPG 101600Z 27006KT 9999 BKN030 BKN190 ...,49.015,2.534,107,"Paris/De Gaulle Arpt, ID, FR",BKN,"[{'cover': 'BKN', 'base': 3000}, {'cover': 'BK...",MVFR,2025-12-10 16:20:04.386928+00:00
4,a2e5ec,SKW4128,United States,1.765380e+09,1765379869,-122.9854,44.2956,3444.24,False,171.99,...,METAR,METAR LFPG 101600Z 27006KT 9999 BKN030 BKN190 ...,49.015,2.534,107,"Paris/De Gaulle Arpt, ID, FR",BKN,"[{'cover': 'BKN', 'base': 3000}, {'cover': 'BK...",MVFR,2025-12-10 16:20:04.386928+00:00


In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10357 entries, 0 to 10356
Data columns (total 39 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   icao24                10357 non-null  object 
 1   callsign              10175 non-null  object 
 2   origin_country        10357 non-null  object 
 3   time_position         10251 non-null  float64
 4   last_contact          10357 non-null  int64  
 5   longitude             10251 non-null  float64
 6   latitude              10251 non-null  float64
 7   baro_altitude         9356 non-null   float64
 8   on_ground             10357 non-null  bool   
 9   velocity              10356 non-null  float64
 10  true_track            10357 non-null  float64
 11  vertical_rate         9382 non-null   float64
 12  sensors               0 non-null      float64
 13  geo_altitude          9258 non-null   float64
 14  squawk                5973 non-null   float64
 15  spi                

## Data Overview

To better understand the dataset and check whether it contains a lot of missing values, we first used the `df.info()`command.

This allow us to draw some initial conclusions that will help us with the next step which is transforming and cleaning data.

In this case, we can see that most of the flight-related variables are well populated.

That means that the dataset contains enough useful information to perform meaningful analysis without heavy data loss early on.

In [5]:
missing_rate = df.isna().mean().sort_values(ascending=False)
missing_rate

sensors                 1.000000
squawk                  0.423289
geo_altitude            0.106112
baro_altitude           0.096650
vertical_rate           0.094139
callsign                0.017573
longitude               0.010235
time_position           0.010235
latitude                0.010235
velocity                0.000097
icao24                  0.000000
last_contact            0.000000
origin_country          0.000000
true_track              0.000000
on_ground               0.000000
spi                     0.000000
position_source         0.000000
fetch_time_utc          0.000000
metar_icaoId            0.000000
metar_receiptTime       0.000000
metar_obsTime           0.000000
metar_reportTime        0.000000
metar_temp              0.000000
metar_dewp              0.000000
metar_wdir              0.000000
metar_wspd              0.000000
metar_visib             0.000000
metar_altim             0.000000
metar_qcField           0.000000
metar_metarType         0.000000
metar_rawO

In [6]:
df = df.drop(columns='sensors')
df.shape

(10357, 38)

In [7]:
df["squawk"].value_counts(dropna=False).head(10)

squawk
NaN       4384
1000.0     434
1200.0      88
7000.0      64
2000.0       9
2657.0       7
4000.0       6
3277.0       6
543.0        6
2630.0       6
Name: count, dtype: int64

In [8]:
df = df.drop(columns="squawk")
df.shape

(10357, 37)

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10357 entries, 0 to 10356
Data columns (total 37 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   icao24                10357 non-null  object 
 1   callsign              10175 non-null  object 
 2   origin_country        10357 non-null  object 
 3   time_position         10251 non-null  float64
 4   last_contact          10357 non-null  int64  
 5   longitude             10251 non-null  float64
 6   latitude              10251 non-null  float64
 7   baro_altitude         9356 non-null   float64
 8   on_ground             10357 non-null  bool   
 9   velocity              10356 non-null  float64
 10  true_track            10357 non-null  float64
 11  vertical_rate         9382 non-null   float64
 12  geo_altitude          9258 non-null   float64
 13  spi                   10357 non-null  bool   
 14  position_source       10357 non-null  int64  
 15  fetch_time_utc     

In [12]:
df["time_position_dt"] = pd.to_datetime(df["time_position"], unit="s", utc=True)
df["last_contact_dt"] = pd.to_datetime(df["last_contact"], unit="s", utc=True)
df["metar_obsTime_dt"] = pd.to_datetime(df["metar_obsTime"], unit="s", utc=True)

snapshot_time_utc = df["last_contact_dt"].max()

snapshot_time_utc

Timestamp('2025-12-10 15:17:50+0000', tz='UTC')

In [14]:
df[[
    "time_position_dt",
    "last_contact_dt",
    "metar_obsTime_dt"
]].describe()

,time_position_dt,last_contact_dt,metar_obsTime_dt
count,10251,10357,10357
mean,2025-12-10 15:17:19.119695360+00:00,2025-12-10 15:17:28.874674176+00:00,2025-12-10 16:00:00+00:00
min,2025-12-10 10:10:30+00:00,2025-12-10 15:12:35+00:00,2025-12-10 16:00:00+00:00
25%,2025-12-10 15:17:47+00:00,2025-12-10 15:17:48+00:00,2025-12-10 16:00:00+00:00
50%,2025-12-10 15:17:49+00:00,2025-12-10 15:17:49+00:00,2025-12-10 16:00:00+00:00
75%,2025-12-10 15:17:49+00:00,2025-12-10 15:17:49+00:00,2025-12-10 16:00:00+00:00
max,2025-12-10 15:17:50+00:00,2025-12-10 15:17:50+00:00,2025-12-10 16:00:00+00:00
